In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

In [2]:
data = pd.read_csv("./data/AAPL/augmentedStockData.csv")

In [3]:
inputs_column_set = ["normDOW", "normDOM", "normDOY", "PercentP1D", "PercentP5D", "PercentP30D"]
labels_column_set = ["PercentF5D"]
period_start = 2588
period_end = 10098
inputs = data[inputs_column_set][period_start:period_end]
labels = data[labels_column_set][period_start:period_end]

In [4]:
train_test_split = 0.85
total_length = len(inputs)
split_value = int(train_test_split * total_length)
training_inputs = inputs[:split_value]
training_labels = labels[:split_value]
testing_inputs = inputs[split_value:]
testing_labels = inputs[split_value:]
training_length = len(training_inputs)
testing_length = len(testing_inputs)
number_of_inputs = training_inputs.shape[1]
number_of_outputs = training_labels.shape[1]
time_steps = 32

In [5]:
def expandTimeSteps(data, time_steps):
    newarr = np.zeros((data.shape[0] - time_steps, time_steps, data.shape[1]))
    for i in range(data.shape[0] - time_steps):
        newarr[i, :, :] = data.iloc[i:i+32]
    return newarr

training_inputs = expandTimeSteps(training_inputs, time_steps)
training_labels = training_labels[:training_inputs.shape[0]]
testing_inputs = expandTimeSteps(testing_inputs, time_steps)
testing_labels = testing_labels[:testing_inputs.shape[0]]

In [8]:
model = Sequential()

model.add(LSTM(2, input_shape=(time_steps,number_of_inputs), activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(4, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(number_of_outputs, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 2)                 72        
_________________________________________________________________
dropout_2 (Dropout)          (None, 2)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 12        
_________________________________________________________________
dropout_3 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 89
Trainable params: 89
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(training_inputs, training_labels, batch_size=32, epochs=20, validation_data=(testing_inputs, testing_labels))

Epoch 1/20
199/199 [==============================] - 2s 5ms/step - loss: 0.0638 - accuracy: 0.0060 - val_loss: 0.1663 - val_accuracy: 0.0317
Epoch 2/20
199/199 [==============================] - 1s 4ms/step - loss: 0.0064 - accuracy: 0.0083 - val_loss: 0.1743 - val_accuracy: 0.0317
Epoch 3/20
199/199 [==============================] - 1s 4ms/step - loss: 0.0044 - accuracy: 0.0068 - val_loss: 0.1767 - val_accuracy: 0.0317
Epoch 4/20
199/199 [==============================] - 1s 4ms/step - loss: 0.0044 - accuracy: 0.0055 - val_loss: 0.1765 - val_accuracy: 0.0317
Epoch 5/20
199/199 [==============================] - 1s 4ms/step - loss: 0.0041 - accuracy: 0.0065 - val_loss: 0.1770 - val_accuracy: 0.0317
Epoch 6/20
199/199 [==============================] - 1s 4ms/step - loss: 0.0042 - accuracy: 0.0063 - val_loss: 0.1749 - val_accuracy: 0.0317
Epoch 7/20
199/199 [==============================] - 1s 4ms/step - loss: 0.0041 - accuracy: 0.0070 - val_loss: 0.1778 - val_accuracy: 0.0317
Epoch 